In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier

In [8]:
input_data = pd.read_csv("train.csv", index_col='PassengerId')

def set_age(name):
    if(name.find('Mr.') != -1): #male - married or not
        return 30
    elif(name.find('Mrs.') != -1): #female - married
        return 45
    elif(name.find('Ms.') != -1): #female - married or not 
        return 30
    elif(name.find('Miss.') != -1): #female - not married 
        return 20
    else:
        return 45
    
max_fare = input_data['Fare'].max()
min_fare = input_data['Fare'].min()
print(max_fare, min_fare)

def clean_dataset(dataset):
    popular_port = dataset['Embarked'].dropna().mode()[0] 
    print("This port is most popular and replace all 'NA' to " + popular_port)
    dataset['Embarked'] = dataset['Embarked'].fillna(popular_port) 
    dataset['Companions'] = dataset['SibSp'] + dataset['Parch'] #number of people travelling with
    dataset['Female'] = dataset['Sex'].map({'female': 1, 'male': 0})
    dataset['Male'] = dataset['Sex'].map({'female': 0, 'male': 1})
    #Embarks as one-hot
    embarked_one_hot = pd.get_dummies(dataset['Embarked'], prefix='Embarked') 
    dataset = dataset.join(embarked_one_hot) 
    #Fare intervals -> one-hot
    fares_intervals = pd.cut(x=dataset['Fare'], bins=[0, 50, 100, 200, 500, 1000])
    fares_one_hot = pd.get_dummies(fares_intervals, prefix='Fare')
    dataset = dataset.join(fares_one_hot)
    dataset['Age'] = dataset.apply(
                            lambda row: set_age(row['Name']) if np.isnan(row['Age']) else row['Age'], axis=1)
    
    dataset = dataset.drop(['Cabin', 'Ticket', 'Name', 'SibSp', 'Parch', 'Embarked', 'Sex', 'Fare'], axis=1) #drop useless columns
    return(dataset)
    

train_data = clean_dataset(input_data)
train_data.to_csv('train_data_results.csv', index=False)
print("Prepared data:")
train_data

This port is most popular and replace all 'NA' to S
Prepared data:


,Survived,Pclass,Sex,Age,Fare,Companions,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,0,3,0,22.0,7.2500,1,0,0,1
2,1,1,1,38.0,71.2833,1,1,0,0
3,1,3,1,26.0,7.9250,0,0,0,1
4,1,1,1,35.0,53.1000,1,0,0,1
5,0,3,0,35.0,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...
887,0,2,0,27.0,13.0000,0,0,0,1
888,1,1,1,19.0,30.0000,0,0,0,1
889,0,3,1,20.0,23.4500,3,0,0,1


In [9]:
 # X and Y ('survived;) axis
 X = train_data.drop(['Survived'], axis=1).values.astype(float)

 Y = train_data['Survived'].values

 def create_model(optimizer='adam', init='uniform'):
    #create model
    model = keras.Sequential()
    model.add( keras.layers.Dense(16, input_dim=X.shape[1], kernel_initializer=init, activation='relu') )
    model.add( keras.layers.Dense(8, kernel_initializer=init, activation='relu') ) 
    model.add( keras.layers.Dense(4, kernel_initializer=init, activation='relu') ) 
    model.add( keras.layers.Dense(1, kernel_initializer=init, activation='sigmoid') )
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print("Model created")
    return model

In [10]:
epochs = 200
batch_size = 5
init = 'glorot_uniform'
optimizer = 'rmsprop'

#create classfier
model_predictions = KerasClassifier(
    model=create_model, optimizer=optimizer, init=init, epochs=epochs,
    batch_size=batch_size)

model_predictions.fit(X,Y)

Model created
Epoch 1/200
179/179 [==============================] - 1s 964us/step - loss: 0.6235 - accuracy: 0.6667
Epoch 2/200
179/179 [==============================] - 0s 983us/step - loss: 0.5926 - accuracy: 0.6891
Epoch 3/200
179/179 [==============================] - 0s 973us/step - loss: 0.5860 - accuracy: 0.6891
Epoch 4/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5856 - accuracy: 0.6914
Epoch 5/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5796 - accuracy: 0.7037
Epoch 6/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5685 - accuracy: 0.7071
Epoch 7/200
179/179 [==============================] - 0s 932us/step - loss: 0.5544 - accuracy: 0.7160
Epoch 8/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5432 - accuracy: 0.7385
Epoch 9/200
179/179 [==============================] - 0s 985us/step - loss: 0.5481 - accuracy: 0.7284
Epoch 10/200
179/179 [==============================] - 0s 1ms/step

179/179 [==============================] - 0s 947us/step - loss: 0.4052 - accuracy: 0.8260
Epoch 159/200
179/179 [==============================] - 0s 889us/step - loss: 0.4005 - accuracy: 0.8305
Epoch 160/200
179/179 [==============================] - 0s 871us/step - loss: 0.3960 - accuracy: 0.8159
Epoch 161/200
179/179 [==============================] - 0s 917us/step - loss: 0.4010 - accuracy: 0.8260
Epoch 162/200
179/179 [==============================] - 0s 911us/step - loss: 0.4008 - accuracy: 0.8294
Epoch 163/200
179/179 [==============================] - 0s 908us/step - loss: 0.4045 - accuracy: 0.8238
Epoch 164/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4102 - accuracy: 0.8272
Epoch 165/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4017 - accuracy: 0.8193
Epoch 166/200
179/179 [==============================] - 0s 972us/step - loss: 0.3995 - accuracy: 0.8294
Epoch 167/200
179/179 [==============================] - 0s 886us/step - 

KerasClassifier(
	model=<function create_model at 0x12b0c8dc0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=200
	init=glorot_uniform
	class_weight=None
)

In [12]:
#predictions for one person
d = {
     'Pclass': [1],    # Ticket class -> 1 = 1st, 2 = 2nd, 3 = 3rd
     'Name': 'Leo',    # Doesn't matter 
     'Sex': 'male',    # male / female
     'Age': [20],      # in years 
     'SibSp': [1],     # Number of siblings / spouses aboard the Titanic
     'Parch': [0],     # Number of parents / children aboard the Titanic
     'Ticket':[0],     # Doesn't matter  
     'Fare': [57],     # 0 - 1000
     'Cabin': [0],     # Doesn't matter 
     'Embarked':'Q',   # Port of Embarkation -> C = Cherbourg, Q = Queenstown, S = Southampton
    }
# add columns that will be missing
if(d['Embarked'] == 'C'):
    d['Embarked_Q'] = [0]
    d['Embarked_S'] = [0]
if(d['Embarked'] == 'Q'):
    d['Embarked_C'] = [0]
    d['Embarked_S'] = [0]
if(d['Embarked'] == 'S'):
    d['Embarked_Q'] = [0]
    d['Embarked_C'] = [0]

df = pd.DataFrame(data=d)
df = clean_dataset(df) # prepare data

X_data = df.values.astype(float)

prediction = model_predictions.predict(X_data)
if(prediction):
    print("You survived!")
else:
    print("Better luck next time...")
print(prediction)

1/1 [==============================] - 0s 93ms/step
Better luck next time...
[0]


In [13]:
#predictions using test dataset from kaggle
test_data = pd.read_csv("test.csv", index_col='PassengerId')

max_faret = test_data['Fare'].max()
min_faret = test_data['Fare'].min()
print(max_faret, min_faret)

test_data = clean_dataset(test_data) #prepare data to same format as train data
print(test_data.isnull().sum())  #one row has Fare NAN
test_data = test_data.fillna(10.000)
#print(test_data)
X_test = test_data.values.astype(float)
test_predictions = model_predictions.predict(X_test)
#print(test_predictions)
# create file to see the score on kaggle
submission = pd.DataFrame({
                'PassengerId': test_data.index,
                'Survived': test_predictions,
})
submission.sort_values('PassengerId', inplace=True)
submission.to_csv('submission.csv', index=False)

#prepare same dataframe as test for charts purposes
test_data['Survived'] = test_predictions

test_data.to_csv('test_data_results.csv', index = False)
test_data

This port is most popular and replace all 'NA' to S
84/84 [==============================] - 0s 932us/step


,Pclass,Sex,Age,Fare,Companions,Embarked_C,Embarked_Q,Embarked_S,Survived
PassengerId,,,,,,,,,
892,3,0,34.5,7.8292,0,0,1,0,0
893,3,1,47.0,7.0000,1,0,0,1,0
894,2,0,62.0,9.6875,0,0,1,0,0
895,3,0,27.0,8.6625,0,0,0,1,0
896,3,1,22.0,12.2875,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...
1305,3,0,30.0,8.0500,0,0,0,1,0
1306,1,1,39.0,108.9000,0,1,0,0,1
1307,3,0,38.5,7.2500,0,0,0,1,0
